In [ ]:
import os
from pathlib import Path
#import requests
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from source.visual_genome_meta_data import read_json_to_dict
from source.visual_genome_data import download_image
from source.visual_genome_data import convert_url
from source.visual_genome_meta_data import count_occurrences
from source.visual_genome_meta_data import get_image_meta_data
from source.visual_genome_meta_data import bboxes_from_metadata
from source.visual_genome_meta_data import plot_image_with_multiple_bboxes
from source.visual_genome_to_yolo import create_class_mapping_from_list
from source.visual_genome_to_yolo import save_class_map_to_yaml
from source.visual_genome_to_yolo import read_yaml_to_class_map
from source.visual_genome_to_yolo import convert_single_image_to_yolo
from source.visual_genome_to_yolo import read_yolo_metadata
#from source.visual_genome_to_yolo import visualize_yolo_annotations


In [ ]:
os.listdir()

In [ ]:
root_path = Path('/Users/stephanehess/Documents/CAS_AML/dias_digit_project')
data_path = Path('/Users/stephanehess/Documents/CAS_AML/dias_digit_project/visual_genome_data')
yolo_path = root_path / 'visual_genome_yolo'


In [ ]:
file_path = data_path/'objects.json'
objects = read_json_to_dict(file_path)
#objects_3 = read_json_to_dict('visual_genome_data/objects_3.json')
#objects_5 = read_json_to_dict('visual_genome_data/objects_5.json')

In [ ]:
len(objects)

### The object is a list:

In [ ]:
type(objects)

In [ ]:
len(objects)

### Each element in the list is a dictionary:

In [ ]:
type(objects[0])

### Each such element (dictionary) discribes an image:

In [ ]:
objects[0].keys()

### Each value element of the key 'object' in the dictionary is again a list: 

In [ ]:
type(objects[0]['objects'])

In [ ]:
len(objects[0]['objects'])

In [ ]:
type(objects[0]['image_url'])

In [ ]:
type(objects[0]['image_id'])

### The 'objects' list appears to contain the annotations and object coordinates:

In [ ]:
objects[0]['objects'][0]

In [ ]:
objects[0]['image_id']

In [ ]:

for obj in objects[0]['objects']:
    print(obj['names'])

In [ ]:
obj_list = [obj['names'][0] for obj in objects[0]['objects']]
obj_list

In [ ]:
obj_list.sort()
obj_list

In [ ]:
round_counter = 0

for idx in list(range(len(objects))):
    #print(idx)
    print('\n')
    print('image id:')
    print(objects[idx]['image_id'])
    print('\n')
    print('image url:')
    print(objects[idx]['image_url'])
    print('\n')
    print('objects:')
    print(len(objects[idx]['objects']))
    #print(objects[idx]['objects'][0]['names'])
    #print([objects[idx]['objects'][i]['names'] for i in list(range(len(objects[idx]['objects'])))])
    names = []
    for idx_obj in list(range(len(objects[idx]['objects']))):
        name = objects[idx]['objects'][idx_obj]['names']
        #print(name)
        names.extend(name)
    print(names)  
    round_counter += 1
    if round_counter > 1:
        break

In [ ]:

#file_path = '/Users/stephanehess/Documents/CAS_AML/dias_digit_project/visual_genome_data'
file_path = str(data_path)
im_url = 'https://cs.stanford.edu/people/rak248/VG_100K_2/1.jpg'


In [ ]:
#download_image(im_url, file_path) 

In [ ]:
os.getcwd()

In [ ]:
os.path.dirname(file_path)

### Cave!
Unfortunately, some images seems to be downloadable from https://cs.stanford.edu/people/rak248/VG_100K_2.
Other images the following address has to be used: 
https://cs.stanford.edu/people/rak248/VG_100K
Some of the images on https://cs.stanford.edu/people/rak248/VG_100K have a wrong url attached to them in the objects object: http://crowdfile.blob.core.chinacloudapi.cn/4615/

Consider the example below: 

In [ ]:
objects[0]['image_url']

In [ ]:
objects[101000]['image_url']

In [ ]:
test_url = 'https://cs.stanford.edu/people/rak248/VG_100K/' + '2317239.jpg'
test_url

In [ ]:
converted_url = convert_url(test_url)
converted_url

In [ ]:
convert_url(converted_url)

In [ ]:
objects[101000]['image_id']

In [ ]:
data_path

In [ ]:
str(data_path)

In [ ]:
data_path

In [ ]:
all_file_list = os.listdir(data_path)
image_file_list = []
for filename in all_file_list:
    file_extension = filename.split('.')[-1]
    if file_extension == 'jpg':
        image_file_list.append(filename)
    
print(len(image_file_list))
image_file_list.sort()
image_file_list[0:7]

In [ ]:
image_ids_present = []
for filename in image_file_list:
    img_id = int(filename.split('_')[-1].split('.')[0])
    image_ids_present.append(img_id)
print(len(image_ids_present))
image_ids_present_set = set(image_ids_present)

In [ ]:
desired_objects = ['eins', 'drei']

occurrence_counts = dict.fromkeys(desired_objects, 0)

input_list = ['eins', 'zwei', 'eins', 'drei', 'eins', 'vier']

In [ ]:
occurrence_counts

In [ ]:
count_occurrences(occurrence_counts, input_list)

In [ ]:
#file_path = '/Users/stephanehess/Documents/CAS_AML/dias_digit_project/visual_genome_data'
file_path = str(data_path)
desired_objects = ['bridge', 'lighthouse', 'water tower', 'windmill']
#desired_objects = ['church']
#desired_objects = ['mountain', 'mountains', 'church', 'lake', 'river', 'stream', 'glacier', 'water body']
#desired_objects = ['forest', 'mountain', 'mountains',
#                   'church', 'lake', 'river', 'stream', 'glacier', 'water body', 
#                   'watercourse']


#desired_objects = ['tree', 'man']

#desired_objects = ['car']

failed_downloads = []

chinacloudapi = []

round_counter = 0

image_counter = 0

occurrence_counts = dict.fromkeys(desired_objects, 0)



for idx in list(range(len(objects))): # objects is a list where each element is a dictionary representing one image
    round_counter += 1
    if objects[idx]['image_id'] == 1001:
        continue
    if objects[idx]['image_id'] in image_ids_present_set: # Check if the image is already present among the previously downloaded images
        continue
    if ('image_url' not in list(objects[idx].keys())): # Check if url is present in the meta-data
        continue

    names = [] # Make a list of objects contained in the image (I will use it further down to check if it containes one of my desired objects)
    for idx_obj in list(range(len(objects[idx]['objects']))):
        name = objects[idx]['objects'][idx_obj]['names']
        #print(name)
        names.extend(name)
        
    inter_set = set(desired_objects).intersection(set(names)) # Check if the image in question contains any of the objects I am interested in
    #print('\n')
    #print(inter_set)
    if len(inter_set) == 0: 
        continue
    
    
    if 'chinacloudapi' in objects[idx]['image_url']: # Check if the url needs to be adapted (the chinacloudapi seems to not work anymore)
        image_id = objects[idx]['image_id']
        im_url = 'https://cs.stanford.edu/people/rak248/VG_100K/' + str(image_id) + '.jpg'
        chinacloudapi.append(objects[idx]['image_url'])
    else:
        im_url = objects[idx]['image_url'] 

    down_load_bool = download_image(im_url, file_path, change_url=True) 
    if down_load_bool:
        print('change_url=True worked')
        count_occurrences(occurrence_counts, names)
        image_counter += 1
    else:
        down_load_bool = download_image(im_url, file_path, change_url=False) 
        if down_load_bool:
            count_occurrences(occurrence_counts, names)
            image_counter += 1
        else:
            failed_downloads.append(im_url)
            continue
        
    
    
    
    
    #print('round_counter')
    #print(round_counter)
    #if round_counter > 1:
    #   break
    
    
    #if len(inter_set) == 1:
    #    print(round_counter)
    #    print(inter_set)
    #    break



### Download images based on the objects they contain:

In [ ]:
#file_path = '/Users/stephanehess/Documents/CAS_AML/dias_digit_project/visual_genome_data'
file_path = str(data_path)
desired_objects = ['church']
#desired_objects = ['mountain', 'mountains', 'church', 'lake', 'river', 'stream', 'glacier', 'water body']
#desired_objects = ['forest', 'mountain', 'mountains',
#                   'church', 'lake', 'river', 'stream', 'glacier', 'water body', 
#                   'watercourse']

#desired_objects = ['tree', 'man']

#desired_objects = ['car']

failed_downloads = []

chinacloudapi = []

round_counter = 0

image_counter = 0

occurrence_counts = dict.fromkeys(desired_objects, 0)



for idx in list(range(len(objects))): # objects is a list where each element is a dictionary representing one image

    if objects[idx]['image_id'] == 1001:
        continue
    if objects[idx]['image_id'] in image_ids_present_set: # Check if the image is already present among the previously downloaded images
        continue

    names = [] # Make a list of objects contained in the image (I will use it further down to check if it containes one of my desired objects)
    for idx_obj in list(range(len(objects[idx]['objects']))):
        name = objects[idx]['objects'][idx_obj]['names']
        #print(name)
        names.extend(name)

    
    if ('image_url' not in list(objects[idx].keys())): # Check if url is present in the meta-data
        continue
    else:
        if 'chinacloudapi' in objects[idx]['image_url']: # Check if the url needs to be adapted (the chinacloudapi seems to not work anymore)
            image_id = objects[idx]['image_id']
            im_url = 'https://cs.stanford.edu/people/rak248/VG_100K/' + str(image_id) + '.jpg'
            chinacloudapi.append(objects[idx]['image_url'])
        else: 
            im_url = objects[idx]['image_url']

        inter_set = set(desired_objects).intersection(set(names)) # Check if the image in question contains any of the objects I am interested in
        #print('\n')
        #print(inter_set)
        if len(inter_set) > 0: 
           #print('\n')
           #print('image id:')
           #print(objects[idx]['image_id'])
           #print('\n')
           #print('inter_set:')
           #print(inter_set)
           #print(len(inter_set))
           #print('names')
           #print(names)
           #print('\n')
           #print('image url:')
           #print(im_url)
            down_load_bool = download_image(im_url, file_path, change_url=True) 
            if down_load_bool:
                count_occurrences(occurrence_counts, names)
                image_counter += 1
            else:
                down_load_bool = download_image(im_url, file_path, change_url=False) 
                if down_load_bool:
                    count_occurrences(occurrence_counts, names)
                    image_counter += 1
                else:
                    failed_downloads.append(im_url)
                    continue
            
        
    round_counter += 1
    #print('round_counter')
    #print(round_counter)
    #print(round_counter > 7)
    #if round_counter > 10:
    #    break

## Count objects contained in downloaded images: 

In [ ]:
occurrence_counts

In [ ]:
image_counter

In [ ]:
idx

In [ ]:
idx

In [ ]:
len(failed_downloads)

len(failed_downloads)
1

In [ ]:
failed_downloads

In [ ]:
len(chinacloudapi)

In [ ]:
chinacloudapi[0:7]

In [ ]:
objects[1005]['image_id']

## Quickly test the object counting method:

In [ ]:
#file_path = '/Users/stephanehess/Documents/CAS_AML/dias_digit_project/visual_genome_data'
file_path = str(data_path)

#desired_objects = ['tree', 'man']

#desired_objects = ['car']
#desired_objects = ['church']
desired_objects = ['bridge', 'lighthouse', 'water tower', 'windmill']

round_counter = 0

image_counter = 0

occurrence_counts = dict.fromkeys(desired_objects, 0)



for idx in list(range(len(objects))):

    #print('\n')
    #print(idx)
    #print('\n')
    #print('image id:')
    #print(objects[idx]['image_id'])
    if objects[idx]['image_id'] == 1001:
        continue

    #print('\n')
    #print('objects:')
    #print(len(objects[idx]['objects']))
    #print(objects[idx]['objects'])
    #print(objects[idx]['objects'][0]['names'])
    #print([objects[idx]['objects'][i]['names'] for i in list(range(len(objects[idx]['objects'])))])
    names = []
    for idx_obj in list(range(len(objects[idx]['objects']))):
        name = objects[idx]['objects'][idx_obj]['names']
        #print(name)
        names.extend(name)
    #print(names)  
    #print('trees' in names)
    
    if ('image_url' not in list(objects[idx].keys())):
        continue
    #elif 'chinacloudapi' in objects[idx]['image_url']:
    #    #print('hey')
    #    continue
    else:
        im_url = objects[idx]['image_url']

        #if ('mountain' in names) and ('chinacloudapi' not in im_url):


        #count_occurrences(occurrence_counts, names)
        inter_set = set(desired_objects).intersection(set(names))
        #print('\n')
        #print(inter_set)
        if len(inter_set) > 0: 
           #print('\n')
           #print('image id:')
           #print(objects[idx]['image_id'])
           #print('\n')
           #print('inter_set:')
           #print(inter_set)
           #print(len(inter_set))
           #print('names')
           #print(names)
           #print('\n')
           #print('image url:')
           #print(im_url)
            count_occurrences(occurrence_counts, names)
            image_counter += 1
            
        
    round_counter += 1
    #print('round_counter')
    #print(round_counter)
    #print(round_counter > 7)
    #if round_counter > 0:
     #   break

In [ ]:
occurrence_counts

In [ ]:

image_number = 0
print(len(objects[image_number]['objects']))
#print(objects[image_number]['objects'][0]['names'])

object_name_list = []
for idx in list(range(0, len(objects[image_number]['objects']))):
    #print(idx)
    names = objects[image_number]['objects'][idx]['names']
    #print(names)
    object_name_list.extend(names)

tree_counter = 0
man_counter = 0
car_counter = 0
for obj in object_name_list:
    if obj == 'tree':
        tree_counter +=1
    if obj == 'man':
        man_counter += 1
    if obj == 'car':
        car_counter += 1

In [ ]:
print(tree_counter)
print(man_counter)
print(car_counter)

In [ ]:
object_name_list.sort()
object_name_list

In [ ]:
objects[0]['objects'][0:3]

In [ ]:
for obj in objects[0]['objects']:
    print(obj['names'])

In [ ]:
idx

### Quickly test the create_class_mapping_from_list function: 

In [ ]:
desired_objects = ['tree', 'man']
#desired_objects = ['car']
desired_objects

In [ ]:
class_map = create_class_mapping_from_list(desired_objects)

In [ ]:
desired_objects

In [ ]:
class_map

### Quickly test the save_class_map_to_yaml and read_yaml_to_class_map functions: 

In [ ]:
os.getcwd()

In [ ]:
class_map

In [ ]:
#file_path = '/Users/stephanehess/Documents/CAS_AML/dias_digit_project/visual_genome_yolo/'
file_path = str(yolo_path) + '/'

output_path = file_path + 'class_map_test.yaml'
save_class_map_to_yaml(class_map, output_path)


In [ ]:
file_list = os.listdir(yolo_path)
print(file_list)
for filename in file_list:
    if filename.split('_')[-1] == 'test.yaml':
        print(filename)
        yaml_file_name = filename
yaml_file_name

In [ ]:

yaml_path = yolo_path/yaml_file_name

class_map = read_yaml_to_class_map(str(yaml_path))

# Print the class mapping
print(class_map)

### Quickly test convert_single_image_to_yolo function and associated workflow:

In [ ]:
img_id = 23

In [ ]:
label_path_gen = yolo_path/'visual_genome_'
label_path = str(label_path_gen) + str(img_id) + '.txt'
print(label_path)

image_path_gen = data_path/'visual_genome_'
image_path = str(image_path_gen) + str(img_id) + '.jpg'
print(image_path)

yaml_path = yolo_path/'class_map_test.yaml'
yaml_path = str(yaml_path)
print(yaml_path)

In [ ]:
img_data = get_image_meta_data(objects, img_id)
img_data['image_id']

In [ ]:
object_name_list = []
for idx in list(range(0, len(img_data['objects']))):
    #print(idx)
    names = img_data['objects'][idx]['names']
    #print(names)
    object_name_list.extend(names)
print(object_name_list)

In [ ]:
desired_object = 'tree'

In [ ]:

list_of_boxes = bboxes_from_metadata(image_path, objects, desired_object)

list_of_boxes

In [ ]:
list_of_boxes

In [ ]:
#Example usage:
#bboxes = [(0.5, 0.5, 0.3, 0.4), (0.7, 0.7, 0.2, 0.2)]
#bboxes = [(x_center, y_center, width, height), (0.7, 0.7, 0.2, 0.2)]
bboxes = list_of_boxes

class_names = ['desired_object', 'desired_object']
plot_image_with_multiple_bboxes(image_path, bboxes, class_names)

### Quickly test convert_single_image_to_yolo function:

In [ ]:
image_dir = str(data_path) + '/'
output_dir = str(yolo_path)


#label_path = convert_single_image_to_yolo(objects[0], class_map, image_dir, output_dir)
label_path = convert_single_image_to_yolo(img_data, class_map, image_dir, output_dir)


In [ ]:
label_path

In [ ]:
yolo_images = os.listdir(yolo_path)
yolo_images

In [ ]:
data_path

In [ ]:
class_map

In [ ]:
yaml_path

In [ ]:
labels, bboxes = read_yolo_metadata(label_path, class_map)

In [ ]:
labels

In [ ]:
len(bboxes)

In [ ]:
class_names = list(labels)
plot_image_with_multiple_bboxes(image_path, bboxes, class_names)

In [ ]:
def transform_to_yolo_format(img_data, class_map, img_width=None, img_height=None):
    """
    Transform Visual Genome object data to YOLO format.
    
    Args:
        img_data: Dictionary containing Visual Genome image metadata
        class_map: Dictionary mapping class names to class IDs
        img_width: Image width (if None, tries to extract from img_data)
        img_height: Image height (if None, tries to extract from img_data)
    
    Returns:
        tuple: (yolo_objects, human_readable_objects)
            - yolo_objects: List of lists with [class_id, x_center, y_center, width, height]
            - human_readable_objects: List of dicts with class name and normalized coordinates
    """
    # If image dimensions are not provided, try to infer them
    if img_width is None or img_height is None:
        # Try to get from image dimensions if available
        # Otherwise use max coordinates from objects as approximate dimensions
        max_x = max_y = 0
        for obj in img_data['objects']:
            max_x = max(max_x, obj['x'] + obj['w'])
            max_y = max(max_y, obj['y'] + obj['h'])
        
        img_width = img_width or max_x
        img_height = img_height or max_y
    
    # Lists to store the results
    yolo_objects = []
    human_readable_objects = []
    
    # Process each object
    for obj in img_data['objects']:
        # Get object class
        if isinstance(obj['names'], list) and len(obj['names']) > 0:
            class_name = obj['names'][0]
        elif isinstance(obj['names'], str):
            class_name = obj['names']
        else:
            continue  # Skip objects with no name
        
        # Skip if class not in mapping
        if class_name not in class_map:
            continue
        
        # Get class ID
        class_id = class_map[class_name]
        
        # Get bounding box coordinates
        x, y = obj['x'], obj['y']
        w, h = obj['w'], obj['h']
        
        # Skip invalid bounding boxes
        if w <= 0 or h <= 0:
            continue
        
        # Convert to YOLO format (normalized center coordinates)
        x_center = (x + w/2) / img_width
        y_center = (y + h/2) / img_height
        width = w / img_width
        height = h / img_height
        
        # Ensure coordinates are within [0, 1]
        x_center = max(0, min(1, x_center))
        y_center = max(0, min(1, y_center))
        width = max(0, min(1, width))
        height = max(0, min(1, height))
        
        # Add to YOLO objects list (format used in text files)
        yolo_objects.append([class_id, x_center, y_center, width, height])
        
        # Add to human-readable list
        human_readable_objects.append({
            'class_name': class_name,
            'class_id': class_id,
            'original': {
                'x': x,
                'y': y,
                'width': w,
                'height': h
            },
            'yolo_normalized': {
                'x_center': x_center,
                'y_center': y_center,
                'width': width,
                'height': height
            }
        })
    
    return yolo_objects, human_readable_objects


In [ ]:
# Helper function to format YOLO objects as a string (same as in the text file)
def format_yolo_objects_as_string(yolo_objects):
    """Convert YOLO objects to the string format used in label files."""
    lines = []
    for obj in yolo_objects:
        class_id, x_center, y_center, width, height = obj
        lines.append(f"{class_id} {x_center} {y_center} {width} {height}")
    return "\n".join(lines)

# Example usage:
# img_data = {...}  # Your Visual Genome image data
# class_map = {'tree': 0, 'man': 1}  # Your class mapping
# yolo_objects, human_readable = transform_to_yolo_format(img_data, class_map)
# 
# # Get the text file content
# yolo_text = format_yolo_objects_as_string(yolo_objects)
# print(yolo_text)
# 
# # Print human-readable version
# import json
# print(json.dumps(human_readable, indent=2))

In [ ]:
def visualize_yolo_annotations(image_path, label_path, yaml_path):
    """
    Visualize an image with its YOLO format bounding boxes.
    
    Args:
        image_path: Path to the image file
        label_path: Path to the YOLO format text file with bounding boxes
        yaml_path: Path to the YAML file with class mappings
    """
    # Use the helper function to get class mapping
    class_map = read_yaml_to_class_map(yaml_path)
    
    # Convert class_map to a list where index is class_id
    class_names = list(class_map.keys())
    
    print(f"Label path: {label_path}")
    print(f"Image path: {image_path}")
    
    # Load the image
    image = Image.open(image_path)
    img_width, img_height = image.size
    
    # Convert PIL Image to numpy array for matplotlib
    image_np = np.array(image)
    
    # Create a figure and axis
    fig, ax = plt.subplots(1, figsize=(12, 9))
    
    # Display the image
    ax.imshow(image_np)
    
    # Define colors for bounding boxes
    cmap = plt.cm.get_cmap('hsv', len(class_names))

In [ ]:
label_path = yolo_path/'visual_genome_1.txt'
label_path = str(image_path)

image_path = data_path/'visual_genome_1.jpg'
image_path = str(image_path)

yaml_path = yolo_path/'class_map_test.yaml'
yaml_path = str(yaml_path)

visualize_yolo_annotations(image_path, label_path, yaml_path)

In [ ]:
class_map

In [ ]:
#image_dir = '/Users/stephanehess/Documents/CAS_AML/dias_digit_project/visual_genome_data/'
image_dir = data_path
#output_dir = '/Users/stephanehess/Documents/CAS_AML/dias_digit_project/visual_genome_yolo'
output_dir = yolo_path
label_path = convert_single_image_to_yolo(objects[0], class_map, image_dir, output_dir)


In [ ]:
image_dir = '/Users/stephanehess/Documents/CAS_AML/dias_digit_project/visual_genome_data/'
yolo_dir = '/Users/stephanehess/Documents/CAS_AML/dias_digit_project/visual_genome_yolo/'

image_path = image_dir + 'visual_genome_1.jpg'
bbox_path = yolo_dir + 'visual_genome_1.txt'
yaml_path = yolo_dir + 'class_map.yaml'



In [ ]:
class_map = read_yaml_to_class_map(yaml_path)
class_map

In [ ]:
print(image_path)
print(bbox_path)
print(yaml_path)

In [ ]:
os.getcwd()

In [ ]:
image_dir = '/Users/stephanehess/Documents/CAS_AML/dias_digit_project/visual_genome_data/'
output_dir = '/Users/stephanehess/Documents/CAS_AML/dias_digit_project/visual_genome_yolo'
label_path = convert_single_image_to_yolo(objects[0], class_map, image_dir, output_dir)


In [ ]:
all_files = os.listdir('visual_genome_data')
image_file_list = [f for f in all_files if f.lower().endswith('.jpg')]
image_file_list.sort()
image_file_list[0:5]

In [ ]:
round_counter = 0
for filename in image_file_list:
    file_path = image_dir + filename
    print(file_path)
    round_counter += 1
    if round_counter > 5: 
        break

In [ ]:

round_counter = 0
for filename in image_file_list:
    file_path = image_dir + filename
    print(file_path)
    last_part = file_path.split('_')[-1]
    img_id_str = last_part.split('.')[0]
    img_id = int(img_id_str)
    img_id_minus_1 = img_id-1
    print(img_id)
    print(objects[img_id_minus_1]['image_id'])
    #
    #image = Image.open(file_path)
    ## Convert PIL Image to numpy array for matplotlib
    #image_np = np.array(image)
    #plt.imshow(image_np)
    #plt.show()
    label_path = convert_single_image_to_yolo(objects[img_id_minus_1], class_map, image_dir, output_dir)
    print(label_path)
    round_counter += 1
    #if round_counter > 5: 
     #   break

In [ ]:
import cv2
import numpy as np
import yaml
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import os

def visualize_yolo_annotations(image_path, label_path, yaml_path):
    """
    Visualize an image with its YOLO format bounding boxes.
    
    Args:
        image_path: Path to the image file
        label_path: Path to the YOLO format text file with bounding boxes
        yaml_path: Path to the YAML file with class mappings
    """
    # Load the class mappings from YAML file
    with open(yaml_path, 'r') as f:
        yaml_data = yaml.safe_load(f)
    
    class_names = yaml_data['names']
    print(label_path)
    # Load the image
    print(image_path)
    image = Image.open(image_path)
    img_width, img_height = image.size
    
    # Convert PIL Image to numpy array for matplotlib
    image_np = np.array(image)
    
    # Create a figure and axis
    fig, ax = plt.subplots(1, figsize=(12, 9))
    
    # Display the image
    ax.imshow(image_np)
    
    # Define colors for bounding boxes (one for each class)
    # Using a colormap to generate distinct colors
    cmap = plt.cm.get_cmap('hsv', len(class_names))
    colors = [cmap(i) for i in range(len(class_names))]
    
    # Load and draw bounding boxes from the label file
    with open(label_path, 'r') as f:
        for line in f:
            data = line.strip().split()
            print(data)
            if len(data) != 5:  # YOLO format has 5 values per line
                continue
                
            class_id = int(data[0])
            x_center = float(data[1]) * img_width
            y_center = float(data[2]) * img_height
            width = float(data[3]) * img_width
            height = float(data[4]) * img_height
            
            # Calculate top-left corner for rectangle
            x_min = x_center - (width / 2)
            y_min = y_center - (height / 2)
            
            # Create a rectangle patch
            rect = patches.Rectangle(
                (x_min, y_min), width, height, 
                linewidth=2, 
                edgecolor=colors[class_id], 
                facecolor='none'
            )
            
            # Add the rectangle to the plot
            ax.add_patch(rect)
            
            # Add class label text above the bounding box
            class_name = class_names[class_id] if class_id < len(class_names) else f"Class {class_id}"
            plt.text(
                x_min, y_min - 5, 
                class_name,
                color=colors[class_id], 
                fontsize=12, 
                bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=0)
            )
    
    plt.axis('off')  # Hide axes
    plt.tight_layout()
    
    return fig, ax


In [ ]:
yaml_path

In [ ]:
image_file_list = os.listdir(data_path)
image_file_list[0:5]

In [ ]:

round_counter = 0
for filename in image_file_list:
    image_path = image_dir + filename
    print(image_path)
    last_part = image_path.split('_')[-1]
    img_id_str = last_part.split('.')[0]
    img_id = int(img_id_str)
    img_id_minus_1 = img_id-1
    #print(img_id)
    #print(objects[img_id_minus_1]['image_id'])
    print(image_path.split('.')[0])
    file_path_no_ext = image_path.split('.')[0]
    print(file_path_no_ext)
    label_path_no_ext = file_path_no_ext.replace('visual_genome_data', 'visual_genome_yolo')
    print(label_path_no_ext)
    label_path = label_path_no_ext + '.txt'
    
    visualize_yolo_annotations(image_path, label_path, yaml_path)
    
    
    round_counter += 1
    if round_counter > 5: 
        break

In [ ]:
file_path = '/Users/stephanehess/Documents/CAS_AML/dias_digit_project/visual_genome_data'
desired_objects = ['mountain', 'mountains', 'hill', 'hills', 
                   'church', 'city', 'village', 'lake', 'river', 'stream', 
                   'glacier', 'water body', 'watercourse', 'waters']
round_counter = 0

image_counter = 0

occurrence_counts = dict.fromkeys(desired_objects, 0)



for idx in list(range(len(objects))):
    #print(idx)
    #print('\n')
    #print('image id:')
    #print(objects[idx]['image_id'])
    #print(type(objects[idx]['image_id']))

    #print('\n')
    #print('objects:')
    #print(len(objects[idx]['objects']))
    #print(objects[idx]['objects'])
    #print(objects[idx]['objects'][0]['names'])
    #print([objects[idx]['objects'][i]['names'] for i in list(range(len(objects[idx]['objects'])))])
    names = []
    for idx_obj in list(range(len(objects[idx]['objects']))):
        name = objects[idx]['objects'][idx_obj]['names']
        #print(name)
        names.extend(name)
    #print(names)  
    #print('trees' in names)
    inter_set = set(desired_objects).intersection(set(names))
    if ('image_url' not in list(objects[idx].keys())):
        continue
    else:
        
        im_url = objects[idx]['image_url']
        #if ('mountain' in names) and ('chinacloudapi' not in im_url):


        #count_occurrences(occurrence_counts, names)
        inter_set = set(desired_objects).intersection(set(names))
        if len(inter_set) > 0: 
            print('\n')
            print('image id:')
            print(objects[idx]['image_id'])
            print('\n')
            print('inter_set:')
            print(inter_set)
            print(len(inter_set))
            print('names')
            print(names)
            print('\n')
            print('image url:')
            print(im_url)
            count_occurrences(occurrence_counts, names)
            image_counter += 1
            
        
    round_counter += 1
    #print(round_counter)
    #print(round_counter > 7)
    #if round_counter > 1000:
     #   break

In [ ]:
print(occurrence_counts)

In [ ]:
image_counter

In [ ]:
round_counter

In [ ]:
idx

In [ ]:
'image_url' in list(objects[idx].keys())

In [ ]:
relationships = read_json_to_dict('relationships.json')

In [ ]:
type(relationships)

In [ ]:
len(relationships)

In [ ]:
type(relationships[0])

In [ ]:
len(relationships[0].keys())

In [ ]:
relationships[0].keys()

In [ ]:
len(relationships[0]['relationships'])

In [ ]:
relationships[0]['image_id']

In [ ]:
relationships[2]['image_id']

In [ ]:
len(relationships[2]['relationships'])

In [ ]:
round_counter = 0

for idx in list(range(len(relationships))):
    #print(idx)
    print('\n')
    print('image id:')
    print(relationships[idx]['image_id'])
    print('\n')
    print('relationships:')
    print(len(relationships[idx]['relationships']))
    print(type(relationships[idx]['relationships']))
    print('\n')
    print('relationship 0:')
    print(type(relationships[idx]['relationships'][0]))
    print(relationships[idx]['relationships'][0].keys())
    #print(relationships[idx]['relationships'][0]['subject'])
    print('\n')
    print('relationship 0:')
    print(relationships[idx]['relationships'][0]['object'])
    print(relationships[idx]['relationships'][0]['subject'])
    round_counter += 1
    if round_counter > 3:
        break

In [ ]:
from ultralytics import YOLO